In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
import pandas as pd 

import time

In [3]:
def click_btn(clickable):
    for btn in clickable:
        try:
            btn.click()
        except:
            pass

In [4]:
# startpoint = 0
# count = 1000
# prefix_url = "https://growdiaries.com/explore"

service = Service('./chromedriver.exe')
op = webdriver.ChromeOptions()
op.add_argument('--headless')
# elements = []
# url_df = pd.DataFrame(data={"DiaryName": "", "Url": ""})

In [5]:
# driver = webdriver.Chrome(service=service, options=op)
# while startpoint < 10000:
#     query = f"?action=loadpage&category=all&tags=harvested&start={startpoint}&count={count}&undefined=&ajax=true&ust=9c824c5a63187dd5f579502e26ac3594c3dbb8752fd954432a4d6e1b0ff2e41f8654e9f3"
#     final_url = prefix_url + query
#     driver.get(final_url)
#     elements= driver.find_elements(By.XPATH, "//a[@class='name']")
#     startpoint += count
#     for element in elements:
#         temp_df = pd.DataFrame({"DiaryName": [element.text], "Url": [element.get_attribute('href')]})
#         url_df = pd.concat([url_df, temp_df], ignore_index=True)
# driver.quit()


In [6]:
# url_df.to_csv("Diary Links.csv", index=False)

In [7]:
url_df = pd.read_csv("Diary Links.csv")

In [14]:
currUrl = url_df['Url'].iloc[1]
currUrl

'https://growdiaries.com/diaries/164151-grow-journal-by-jle-bwk'

In [9]:
# driver.get(currUrl)
# driver.maximize_window()
# clickable = driver.find_elements(By.XPATH, "//div[@class='btn_less']")
# click_btn(clickable)

In [10]:
det_df = pd.DataFrame()

In [12]:
driver = webdriver.Chrome(service=service, options=op)
for j, url in enumerate(url_df["Url"]):
    driver.get(url)
    driver.maximize_window()
    clickable = driver.find_elements(By.XPATH, "//div[@class='btn_less']")
    click_btn(clickable)
    report_items = driver.find_elements(By.XPATH, "//div[contains(@class,'report_items')]//*[@class='info']")
    outcome = driver.find_elements(By.XPATH, "//div[contains(@class,'parameters_item')]")
    d = {"diary_name": url_df['DiaryName'].loc[j], "strain": "", "strains_company":"", "light_watt": "", "nutrients": "", "indoor": "", "watering": "", "soil": "", "germination": "", 'grow_techniques': "", "grow_room_size": "",
 "weeks_to_harvest": "", "num_of_plants": "", "bud_dry_weight": "", "bud_wet_weight": ""}
    for i, item in enumerate(report_items):
        text = item.text
        texts = text.split("\n")
        if i == 0:
            d["strain"] = texts[0]
            d['strains_company'] = texts[1]
        elif "LED" in text:
            d['light_watt'] += texts[0] + ","
        elif "Nutrients" in text:
            d['nutrients'] += texts[0] +","
        elif "Room Type" in text:
            d['indoor'] = True
        elif "Watering" in text:
            d['watering'] = texts[0]
        elif "Soil" in text or "Grow" in text:
            d['soil']+= text.replace("\n", " ")+","
        elif "Germination" in text:
            d['germination'] = texts[0]
        elif text.find("Week") > 0 and len(texts[0]) > 1:
            d['grow_techniques'] += texts[0] + ","
        elif text.find("Tent") > 0:
            d['grow_room_size'] = texts[0]
    for i, details in enumerate(outcome):
        text = details.text
        texts = text.split("\n")
        if i == 0:
            d["weeks_to_harvest"] = texts[1]
        elif "BUD WET WEIGHT" in text:
            d['bud_wet_weight']= texts[1]
        elif "BUD DRY WEIGHT" in text:
            d['bud_dry_weight']= texts[1]
        elif "NUMBER OF PLANTS HARVESTED" in text:
            d['num_of_plants'] = texts[1].split(" ")[0]
        elif "TOTAL LIGHT POWER" in text:
            d['light_watt'] = texts[0]
        elif "GROW ROOM" in text:
            d['grow_room_size']= texts[1]
    # temp_df = pd.DataFrame(d)
    det_df = pd.concat([det_df,pd.DataFrame(d, index=[0])], ignore_index=True)
    if j == 1:
        break
driver.quit()


In [13]:
det_df

,diary_name,strain,strains_company,light_watt,nutrients,indoor,watering,soil,germination,grow_techniques,grow_room_size,weeks_to_harvest,num_of_plants,bud_dry_weight,bud_wet_weight
0,#CannapediaGrowersCup4th,Runtz,Zamnesia Seeds,LIGHT,"Advanced Nutrients,General Hydroponics (GHE),",True,Manual,"Soil Grow Medium,Coco Coir Grow Medium,Perlite...",Paper Towel,"Transplant,Transplantation,Defoliation,Repelle...",1,18,1,120,1130
1,Green crack,Green Crack,Crop King Seeds,LIGHT,"Advanced Nutrients,Fox Farm,",True,Manual,"Soil Grow Medium,",Peat Pellet,"LST,",,10,,,


GROW TECHNIQUES: Low Stress Training, High Stress Training, Sea of Green (SoG), Screen of Green (ScrOG), FIMing, Main-Lining, 12-12 From Seed, Defoliation, TOPPING
Room size in square meters, light in watt. bud weight in grams both dry and wet.

In [ ]:

for i, pname in enumerate(report_items):
    text = pname.text
    texts = text.split("\n")
    if i == 0:
        d["strain"] = texts[0]
        d['strains_company'] = texts[1]
    elif "LED" in text:
        d['light_watt'].append(texts[0])
    elif "Nutrients" in text:
        d['nutrients'].append(texts[0])
    elif "Room Type" in text:
        d['indoor'] = True
    elif "Watering" in text:
        d['watering'] = texts[0]
    elif "Soil" in text or "Grow" in text:
        d['soil'].append(text.replace("\n", " "))
    elif "Germination" in text:
        d['germination'] = texts[0]
    elif text.find("Week") > 0 and len(texts[0]) > 1:
        d['grow_techniques'].append(texts[0])
    elif text.find("Tent") > 0:
        d['grow_room_size'] = texts[0]


In [ ]:
for i, pname in enumerate(outcome):
    text = pname.text
    texts = text.split("\n")
    if i == 0:
        d["weeks_to_harvest"] = texts[1]
    elif "BUD WET WEIGHT" in text:
        d['bud_wet_weight']= texts[1]
    elif "BUD DRY WEIGHT" in text:
        d['bud_dry_weight']= texts[1]
    elif "NUMBER OF PLANTS HARVESTED" in text:
        d['num_of_plants'] = texts[1].split(" ")[0]
    elif "TOTAL LIGHT POWER" in text:
        d['light_watt'] = texts[0]
    elif "GROW ROOM" in text:
        d['grow_room_size']= texts[1]
d

{'strain': 'Runtz',
 'strains_company': 'Zamnesia Seeds',
 'light_watt': 'LIGHT',
 'nutrients': ['Advanced Nutrients', 'General Hydroponics (GHE)'],
 'indoor': True,
 'watering': 'Manual',
 'soil': ['Soil Grow Medium', 'Coco Coir Grow Medium', 'Perlite Grow Medium'],
 'germination': 'Paper Towel',
 'grow_techniques': ['Transplant',
  'Transplantation',
  'Defoliation',
  'Repellent',
  'Topping',
  'LST'],
 'grow_room_size': '1',
 'weeks_to_harvest': '18',
 'num_of_plants': '1',
 'bud_dry_weight': '120',
 'bud_wet_weight': '1130'}